In [1]:
!pip install duckdb

In [3]:
import duckdb

In [12]:
allowance_table_path = "data/allowance_backend_table.csv"
allowance_events = "data/allowance_events.json"
payment_schedule = "data/payment_schedule_backend_table.csv"
current_date = '2024-12-03'

In [13]:
allowance_table = duckdb.sql(f"SELECT * FROM '{allowance_table_path}' where status = 'enabled'")
allowance_events = duckdb.sql(f"SELECT * FROM '{allowance_events}'")
payment_schedule = duckdb.sql(f"SELECT * FROM '{payment_schedule}'")

In [14]:
allowance_events

┌──────────────────────────────────────────────┬───────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────┐
│                     user                     │                             event                             │                                 allowance                                 │
│              struct(id varchar)              │         struct("timestamp" timestamp, "name" varchar)         │ struct(scheduled struct(frequency varchar, "day" varchar), amount bigint) │
├──────────────────────────────────────────────┼───────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────┤
│ {'id': 03c5ffe8-6cd2-434b-a1b2-c4006860d94c} │ {'timestamp': 2024-09-21 01:39:34, 'name': allowance.created} │ {'scheduled': {'frequency': weekly, 'day': sunday}, 'amount': 10}         │
│ {'id': 03c5ffe8-6cd2-434b-a1b2-c4006860d94c} │ {'time

In [15]:
allowance_table

┌──────────────────────────────────────┬───────────────┬───────────┬───────────────┬────────────────────────────────┬──────────────────┬─────────┐
│                 uuid                 │ creation_date │ frequency │      day      │           updated_at           │ next_payment_day │ status  │
│               varchar                │     int64     │  varchar  │    varchar    │            varchar             │      int64       │ varchar │
├──────────────────────────────────────┼───────────────┼───────────┼───────────────┼────────────────────────────────┼──────────────────┼─────────┤
│ 30f4e25e-3e37-462e-8c3c-42f24f54350f │    1724838709 │ monthly   │ fifteenth_day │ 2024-10-15T08:00:41.445627563Z │               15 │ enabled │
│ 6da398ad-079d-49b9-8668-6d7ce4d22683 │    1724659829 │ monthly   │ fifteenth_day │ 1724659829                     │               15 │ enabled │
│ 2d30fe2d-6c32-4b8a-a19b-906184f64f62 │    1731309159 │ monthly   │ fifteenth_day │ 1731309159                     │ 

In [21]:
payment_schedule

┌──────────────────────────────────────┬──────────────┐
│               user_id                │ payment_date │
│               varchar                │    int64     │
├──────────────────────────────────────┼──────────────┤
│ 061d8039-b332-40ca-b906-3514b06411c2 │           22 │
│ 0ca92d01-feed-42e5-b90c-299b615f9d3f │           22 │
│ 160bcd2d-3ab0-4a60-9bfa-ccfcc1ee17e4 │           22 │
│ 1649fee7-c424-4fb7-ac84-db36e15e042a │           22 │
│ 1d647703-874d-4744-b4de-6a5212518aa8 │           22 │
│ 2026a912-d221-42bf-9d4b-ebcf1b9ba02c │           22 │
│ 298b9863-9dca-4f31-b35c-7e19ce678e5e │           22 │
│ 2b0e5667-52e4-40cf-a4dd-c0218d6b7574 │           22 │
│ 2bf2d121-9076-450d-a9e3-3fe6c83dba91 │           22 │
│ 2db8f773-9e65-4200-bbf2-edc4994435a4 │           22 │
│                  ·                   │            · │
│                  ·                   │            · │
│                  ·                   │            · │
│ d3df388a-9e07-41d3-bb4d-57266cb9e18f │        

# Finding duplicates in the payment schedule table

In [18]:
#3 duplicates within the table
duckdb.sql("SELECT count(user_id), count(distinct user_id) FROM payment_schedule")


┌────────────────┬─────────────────────────┐
│ count(user_id) │ count(DISTINCT user_id) │
│     int64      │          int64          │
├────────────────┼─────────────────────────┤
│           2139 │                    2126 │
└────────────────┴─────────────────────────┘

In [20]:
duckdb.sql("""select user_id, count(user_id) as qty from payment_schedule group by user_id
           having count(user_id) >1 """)

┌──────────────────────────────────────┬───────┐
│               user_id                │  qty  │
│               varchar                │ int64 │
├──────────────────────────────────────┼───────┤
│ 19d933a6-2810-4c08-b3d6-d68ef3c6eac3 │     2 │
│ 77cdda96-6e5b-47c1-b486-9f06da2c7372 │     2 │
│ 72190913-1208-4333-aaf4-ea7d04dccacd │     2 │
│ d299d804-0d05-48c8-ad62-601d54438e34 │     2 │
│ abf5cf1e-e81a-4b5f-9d5b-649fa10d8f81 │     2 │
│ 42f61042-3d31-42ff-9f1a-508716069464 │     2 │
│ 9221dccf-44f3-446d-b3ce-19f9123ea627 │     2 │
│ d769685e-9046-4ffe-a51d-829ec7f1d877 │     2 │
│ 72e8fe3f-0dd9-44e3-8775-660bacb17310 │     3 │
│ 172fe923-347e-4d0e-ba82-1e9969aaadde │     2 │
│ eab4d50f-2a17-4f07-b462-53be0e54f6c6 │     2 │
│ c091a91b-c6fd-44cb-8ac7-fe3fdbafccea │     2 │
├──────────────────────────────────────┴───────┤
│ 12 rows                            2 columns │
└──────────────────────────────────────────────┘

In [ ]:
duckdb.sql("""

""")